In [1]:
from pyspark.sql import SparkSession, SQLContext
from pyspark.conf import SparkConf
spark = SparkSession.builder\
        .config('spark.jars.packages', 'org.apache.hadoop:hadoop-azure:3.3.1')\
        .getOrCreate()

Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.hadoop#hadoop-azure added as a dependency


:: loading settings :: url = jar:file:/usr/local/spark-3.2.0-bin-hadoop3.2/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


:: resolving dependencies :: org.apache.spark#spark-submit-parent-7a4903a6-7253-409b-a20e-69332ee7eac1;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-azure;3.3.1 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.11 in central
	found com.microsoft.azure#azure-storage;7.0.1 in central
	found com.fasterxml.jackson.core#jackson-core;2.10.5 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found com.microsoft.azure#azure-keyvault-core;1.0.0 in central
	found com.google.guava#guava;27.0-jre in central
	found com.google.guava#failureaccess;1.0 in central
	found com.google.guava#listenablefuture;9999.0-empty-to-avoid-conflict-with-guava in central
	found com.google.code.findbugs#jsr305;3.0.2 in central
	found org.checkerframework#checker-qual;2.5.2 in central
	found com.google.errorprone#error_prone_annotat

In [2]:
azure_application_id = "4413f43e-1232-47a3-b0dd-b224a5189b30"
azure_application_secret = "<secret>"
azure_ad_id = "2fd0bd59-1136-48e1-884a-0938d52022ce"

container_name = "adzd-adl-data"
storage_account_name = "adzdadl"
mount_path = "/mnt/azuredata"

spark.conf.set(f"fs.azure.account.auth.type.{storage_account_name}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{storage_account_name}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{storage_account_name}.dfs.core.windows.net", azure_application_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{storage_account_name}.dfs.core.windows.net", azure_application_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{storage_account_name}.dfs.core.windows.net", f"https://login.microsoftonline.com/{azure_ad_id}/oauth2/token")

In [3]:
file_path = "2014-01-14.ndjson"
single_file_url = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{file_path}"
all_files_url = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/"

In [4]:
import time

def time_measure(f):
    def timed(*args, **kw):
        ts = time.time()
        result = f(*args, **kw)
        te = time.time()
        
        print('func:%r %r took %2.4f sec' % (f.__name__, args, te-ts))
        return result
    return timed

In [5]:
@time_measure
def load_file(url):
    return spark.read.json(url)

@time_measure
def process_file(url):
    return spark.read.json(url)\
        .groupBy(["country", "parameter"])\
        .avg("value")\
        .collect()

In [6]:
load_file(single_file_url)

func:'load_file' ('abfss://adzd-adl-data@adzdadl.dfs.core.windows.net/2014-01-14.ndjson',) took 5.9749 sec


DataFrame[attribution: array<struct<name:string,url:string>>, averagingPeriod: struct<unit:string,value:bigint>, city: string, coordinates: struct<latitude:double,longitude:double>, country: string, date: struct<local:string,utc:string>, location: string, mobile: boolean, parameter: string, sourceName: string, sourceType: string, unit: string, value: double]

In [7]:
process_file(single_file_url)

func:'process_file' ('abfss://adzd-adl-data@adzdadl.dfs.core.windows.net/2014-01-14.ndjson',) took 2.4742 sec


[Row(country='CN', parameter='pm25', avg(value)=170.56666666666666)]

In [32]:
load_file(all_files_url)

func:'load_file' ('abfss://adzd-adl-data@adzdadl.dfs.core.windows.net/',) took 33.9042 sec


DataFrame[attribution: array<struct<name:string,url:string>>, averagingPeriod: struct<unit:string,value:double>, city: string, coordinates: struct<latitude:double,longitude:double>, country: string, date: struct<local:string,utc:string>, location: string, mobile: boolean, parameter: string, sourceName: string, sourceType: string, unit: string, value: double]

In [34]:
process_file(all_files_url)

func:'process_file' ('abfss://adzd-adl-data@adzdadl.dfs.core.windows.net/',) took 51.1043 sec


[Row(country='BG', parameter='so2', avg(value)=9.879365079365082),
 Row(country='AU', parameter='so2', avg(value)=0.0010872294372294372),
 Row(country='IE', parameter='no2', avg(value)=8.439883720930231),
 Row(country='KV', parameter='pm25', avg(value)=65.28571428571429),
 Row(country='BG', parameter='no2', avg(value)=21.479666666666663),
 Row(country='BD', parameter='pm25', avg(value)=122.47619047619048),
 Row(country='FR', parameter='co', avg(value)=389.3953488372093),
 Row(country='HK', parameter='pm10', avg(value)=39.8531746031746),
 Row(country='KZ', parameter='pm25', avg(value)=2.422857142857143),
 Row(country='US', parameter='so2', avg(value)=0.0005622119815668206),
 Row(country='PE', parameter='pm25', avg(value)=16.039591836734694),
 Row(country='IT', parameter='co', avg(value)=831.9141276699031),
 Row(country='BA', parameter='pm25', avg(value)=161.72045454545457),
 Row(country='BM', parameter='pm25', avg(value)=22.0),
 Row(country='HK', parameter='no2', avg(value)=50.915079365